In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [3]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sahilsadaphal","key":"5d4e2d845079fbb88b93751cdd4a160d"}'}

In [5]:
!mkdir -p ~/.kaggle

In [6]:
!cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d zalando-research/fashionmnist

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other


In [9]:
!unzip fashionmnist.zip

Archive:  fashionmnist.zip
  inflating: fashion-mnist_test.csv  
  inflating: fashion-mnist_train.csv  
  inflating: t10k-images-idx3-ubyte  
  inflating: t10k-labels-idx1-ubyte  
  inflating: train-images-idx3-ubyte  
  inflating: train-labels-idx1-ubyte  


In [10]:
train_df=pd.read_csv('fashion-mnist_test.csv')
test_df=pd.read_csv('fashion-mnist_train.csv')

In [19]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(train_df.drop('label', axis=1), train_df['label'], test_size=0.2, random_state=42)

In [22]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
y_train = LabelEncoder().fit_transform(y_train)
y_val = LabelEncoder().fit_transform(y_val)

In [23]:
x_train=torch.tensor(x_train,dtype=torch.float32)
x_val=torch.tensor(x_val,dtype=torch.float32)
y_train=torch.tensor(y_train,dtype=torch.long)
y_val=torch.tensor(y_val,dtype=torch.long)

In [29]:
from torch.utils.data import Dataset, DataLoader
class FashionMNISTDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
    def __len__(self):
        return len(self.images)
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        return image, label

In [30]:
train_dataset=FashionMNISTDataset(x_train,y_train)
val_dataset=FashionMNISTDataset(x_val,y_val)

In [31]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [32]:
epochs=100
learning_rate=0.01

In [42]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network=nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )
    def forward(self,x):
        return self.network(x)

In [43]:
model=NeuralNetwork(x_train.shape[1])

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr= learning_rate)

In [44]:
model=model.to(device)

In [45]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 1.6212780709266663
Epoch: 2 , Loss: 0.813254466176033
Epoch: 3 , Loss: 0.6365203218460083
Epoch: 4 , Loss: 0.5573340625166893
Epoch: 5 , Loss: 0.5034914827942848
Epoch: 6 , Loss: 0.46547371953725813
Epoch: 7 , Loss: 0.4352888033390045
Epoch: 8 , Loss: 0.4114372983574867
Epoch: 9 , Loss: 0.392355677485466
Epoch: 10 , Loss: 0.37741799613833427
Epoch: 11 , Loss: 0.36019370970129966
Epoch: 12 , Loss: 0.3480069290399551
Epoch: 13 , Loss: 0.33477566942572595
Epoch: 14 , Loss: 0.3231243506073952
Epoch: 15 , Loss: 0.31146404981613157
Epoch: 16 , Loss: 0.302643358618021
Epoch: 17 , Loss: 0.2924816011190414
Epoch: 18 , Loss: 0.28452718229591845
Epoch: 19 , Loss: 0.27494548925757406
Epoch: 20 , Loss: 0.26611452509462835
Epoch: 21 , Loss: 0.25891543462872507
Epoch: 22 , Loss: 0.24885056932270527
Epoch: 23 , Loss: 0.24176362639665602
Epoch: 24 , Loss: 0.23647559954226016
Epoch: 25 , Loss: 0.2264820338487625
Epoch: 26 , Loss: 0.21936156904697418
Epoch: 27 , Loss: 0.21249183997511864

In [46]:
model.eval()

NeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [48]:
# evaluation on train
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

1.0


In [49]:
# evaluation on val
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in val_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8515
